In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import random as rdm
import numpy as np
import math
import warnings

warnings.filterwarnings("ignore") # Se desactivan warnings de conversion de datos

#--- Carga de datos ---#
train_data = np.loadtxt('./data/leukemia_train.csv', delimiter=',')
test_data = np.loadtxt('./data/leukemia_test.csv', delimiter=',')
monitor_data = np.empty((0, train_data.shape[1]))  # Inicializa monitor_data como una matriz vacía

#--- Separación de datos para monitoreo ---#
n_monitor = 10
for i in range(n_monitor):
    idx = rdm.randint(0, train_data.shape[0] - 1)              # Determina una fila aleatoria para pasar a monitorio
    monitor_data = np.vstack([monitor_data, train_data[idx]])  # Agrega la fila al monitoreo
    train_data = np.delete(train_data, idx, axis=0)            # Elmina la fila de los datos de train

#--- Separación de la data en entradas y salidas ---#
train_input = train_data[:, :-1]
train_yd = train_data[:, -1].reshape(-1, 1)
monitor_input = monitor_data[:, :-1]
monitor_yd = monitor_data[:, -1].reshape(-1, 1)
test_input = test_data[:, :-1]
test_yd = test_data[:, -1].reshape(-1, 1)

print("Estructura de train_input:  ", train_input.shape)
print("Estructura de train_yd:     ", train_yd.shape)
print("Estructura de monitor_input:", monitor_input.shape)
print("Estructura de monitor_yd:   ", monitor_yd.shape)
print("Estructura de test_input:   ", test_input.shape)
print("Estructura de test_yd:      ", test_yd.shape)

m = train_input.shape[1]
l = 30
suboptimal = math.factorial(m)/(math.factorial(l)*math.factorial(m-l))
print("Numero de pruebas necesarias para sub-optimal search con", l, "features: ", suboptimal)

Estructura de train_input:   (28, 7129)
Estructura de train_yd:      (28, 1)
Estructura de monitor_input: (10, 7129)
Estructura de monitor_yd:    (10, 1)
Estructura de test_input:    (34, 7129)
Estructura de test_yd:       (34, 1)
Numero de pruebas necesarias para sub-optimal search con 30 features:  1.3824893497630198e+83


In [8]:

#---------------------------#
#--- Sección de training ---#
#---------------------------#

#--- Inicialización de parámetros ---#
alpha = 1
beta = 0.1
best_fitness = -1

cant_features_tot = train_input.shape[1]  # Se guarda el valor de las features totales de la matriz original 

#--- Algoritmo de reducción de features ---#
for i in range(train_input.shape[0]):                       # Se recorren todos los patrones
    column_indices = []
    for j in range(train_input.shape[1]):                   # Se recorren todos los bits del patron
        if train_input[i][j] == 1:
            column_indices.append(j)                        # Si el bit es igual a 1 se guarda en el vector de indices
    input_less_features = train_input[:, column_indices]    # Se genera una nueva matriz que sólo tiene las columnas de interés

    cant_features_new = input_less_features.shape[1]        # Se guarda el valor de las features utilizadas en la nueva matriz
    
    if(cant_features_new == 0):
        continue                                            # Saltea la iteración si toda la entrada es únicamente 0s
    else: 
        # Configurar el modelo MLP
        mlp = MLPClassifier(hidden_layer_sizes=(cant_features_new, 50), max_iter=500, random_state=42)

        # Entrenamiento con el modelo MLP
        mlp.fit(input_less_features, train_yd)

        # Nueva matriz de monitoreo con las columnas de interes
        monitor_less_features = monitor_input[:, column_indices]
        
        # Realizar predicciones en los datos de monitoreo
        predictions = mlp.predict(monitor_less_features)

        # Calcular la precisión en los datos de monitoreo
        accuracy = accuracy_score(monitor_yd, predictions)

        # Calcular el valor de fitness
        fitness = alpha * accuracy - beta * (cant_features_new / cant_features_tot)

        #--- Comparación y reemplazo segun el fitness ---#
        if fitness > best_fitness:
            best_features = column_indices
            best_matrix = input_less_features
            best_fitness = fitness
            best_mlp = mlp

print("El mayor valor de fitness conseguido fue:", best_fitness)
print("Para una matriz que considera los índices:", best_features)


El mayor valor de fitness conseguido fue: 0.9997475101697293
Para una matriz que considera los índices: [593, 1130, 1186, 1332, 1753, 2651, 2786, 3168, 3796, 4216, 4765, 4786, 5089, 5743, 5953, 6188, 6804, 6964]


In [9]:
#-----------------------#
#--- Sección de test ---#
#-----------------------#

# Nueva matriz de monitoreo con las columnas de interes
test_less_features = test_input[:, best_features]

# Realizar predicciones en los datos de monitoreo
predictions = best_mlp.predict(test_less_features)

# Calcular la precisión en los datos de monitoreo
accuracy = accuracy_score(test_yd, predictions)

# Calcular el valor de fitness
#fitness_test = alpha * accuracy - beta * (cant_features_new / cant_features_tot)

print("El mayor valor de accuracy conseguido fue:", accuracy)


El mayor valor de accuracy conseguido fue: 0.6176470588235294
